# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [27]:
import nltk #library for natural language toolkit
import pandas as pd
# nltk.download('punkt') ## run in case of punkt and puknkt_tab error when calculcating the frequency
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Andrei\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [13]:
#Importing data from .csv (the train.csv file in this case)

texts  = pd.read_csv('Data/train/train.csv')
texts.head()


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [32]:
#grouping by author
byAuthor = texts.groupby('author')

#frequency by author
wordFreqByAuthor = nltk.probability.ConditionalFreqDist()

for name, group in byAuthor:
    '''
    transforming all the sentences from an author into one string
    helps us get the frequency of a word
    '''
    sentence = group['text'].str.cat(sep = ' ')

    '''
    lowering the string so we don't encounter issues if
    words are written with an capital letter or not
    '''
    sentence = sentence.lower()

    #splitting the sentence into words(tokens)
    tokens = nltk.tokenize.word_tokenize(sentence)

    #caluclate the frequency of every word(token)
    frequency = nltk.FreqDist(tokens)

    #add the frequency for each author to our dictionary
    wordFreqByAuthor[name] = frequency


IndentationError: unexpected indent (2664329130.py, line 7)